<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/project/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import numpy as np
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.utils import compute_class_weight

import sys
sys.path.append('local_modules')

import mlp
import postprocessing_sw
import cooking_inst_mod
import utils

#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
#@title CHOOSE

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset_Past_Only.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]

X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#SPLIT INTO TRAINING, VALIDATION AND TEST SETS
print("\nSplit into training, validation and test sets...")

train_rate = 80
val_rate = 10
train = round(int((dataset.shape[0]*train_rate)/100))
val = round(int((dataset.shape[0]*(train_rate+val_rate))/100))

train_data = X[:train]
train_label = Y[:train]

val_data = X[train+1:val]
val_label = Y[train+1:val]

test_data = X[val+1:]
test_label = Y[val+1:]
print("DATASET SPLITTED")

#COMPUTE CLASS WEIGHT
labels = np.unique(train_label)
classWeight = compute_class_weight('balanced', labels, train_label)
classWeight = dict(zip(labels,classWeight))

#GENERATE MODEL
print("\nGenerate model...")
model = mlp.generate_model_leaky(train_data.shape[-1], n_features)

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001)

#COMPILE MODEL
print("\nCompile model...")
model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])

#EARLY STOPPING
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

#FIT MODEL
print("\nFit model...")
history = model.fit(train_data, train_label, epochs=10, validation_data = (val_data, val_label), batch_size = 128, shuffle = True, class_weight = classWeight, verbose=1, callbacks = [es])

#EVALUATE MODEL
print("\nEvaluate model...")
scores_test = model.evaluate(test_data, test_label, batch_size=128, verbose = 1)
print("Test loss: %.2f%%" % (scores_test[0] * 100))
print("Test accuracy: %.2f%%" % (scores_test[1] * 100))

#CALCULATE PREDICTIONS
print("\nCalculate predictions...")
pred = model.predict_classes(test_data, batch_size=128, verbose=0)
flat_pred = [item for sublist in pred for item in sublist]

#CONFUSION MATRIX BEFORE POST PROCESSING
print("\n\nCompute confusion matrix BEFORE POST PROCESSING...")
utils.compute_metrics(test_label, flat_pred)

#STORE DATETIME
time = []
for i in test_data:
  time.append(i[-5])

In [0]:
#POST PROCESSING WITH SLIDING WINDOWS (MINUTE BY MINUTE)
new_pred = postprocessing_sw.sliding_windows(flat_pred,35)

#CONFUSION MATRIX AFTER POST PROCESSING
print("\n\nCompute NEW confusion matrix AFTER POST PROCESSING...")
utils.compute_metrics(test_label, new_pred)


In [0]:
#COOKING INSTANCE MODALITY

#before post-processing
print("\nCOOKING INSTANCE MODALITY BEFORE POST PROCESSING")
cooking_inst_mod.get_precision_recall_f1(flat_pred,test_label)
#after post-processing
print("\nCOOKING INSTANCE MODALITY AFTER POST PROCESSING")
cooking_inst_mod.get_precision_recall_f1(new_pred,test_label)